<a href="https://colab.research.google.com/github/zyzsdsg/Machine-Learning/blob/main/code/SimpleFFN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler


In [4]:
vix_data = pd.read_csv("/content/drive/MyDrive/VIX/VIX_History.csv")
vix_data["DATE"] = pd.to_datetime(vix_data["DATE"])
vix_data = vix_data.sort_values('DATE')
close_prices = vix_data['CLOSE'].values
vix_data.head()

,DATE,OPEN,HIGH,LOW,CLOSE
0,1990-01-02,17.24,17.24,17.24,17.24
1,1990-01-03,18.19,18.19,18.19,18.19
2,1990-01-04,19.22,19.22,19.22,19.22
3,1990-01-05,20.11,20.11,20.11,20.11
4,1990-01-08,20.26,20.26,20.26,20.26


In [7]:
def create_dataset(data, window_size=10):
    X, y = [], []
    for i in range(len(data) - window_size - 1):
        x_seq = data[i:i+window_size]
        next_day = data[i + window_size]
        today = data[i + window_size - 1]
        label = 1 if next_day > today else 0  # 涨为1，跌为0
        X.append(x_seq)
        y.append(label)
    return np.array(X), np.array(y)

X, y = create_dataset(close_prices, window_size=10)
print(X)
print(y)

[[17.24 18.19 19.22 ... 20.05 24.64 26.34]
 [18.19 19.22 20.11 ... 24.64 26.34 24.18]
 [19.22 20.11 20.26 ... 26.34 24.18 24.16]
 ...
 [14.77 15.37 15.35 ... 19.43 19.1  21.13]
 [15.37 15.35 15.27 ... 19.1  21.13 19.63]
 [15.35 15.27 15.66 ... 21.13 19.63 22.78]]
[0 0 1 ... 0 1 1]


In [8]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [9]:
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

print(X_tensor.shape, y_tensor.shape)

torch.Size([8867, 10]) torch.Size([8867, 1])


In [10]:
import torch.nn as nn

class VIXPredictor(nn.Module):
    def __init__(self):
        super(VIXPredictor, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(10, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

model = VIXPredictor()


In [11]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 简单训练过程
for epoch in range(20):
    model.train()
    optimizer.zero_grad()
    output = model(X_tensor)
    loss = criterion(output, y_tensor)
    loss.backward()
    optimizer.step()

    pred = (output > 0.5).float()
    acc = (pred == y_tensor).float().mean()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}, Accuracy: {acc.item():.4f}")


Epoch 1, Loss: 0.6923, Accuracy: 0.4980
Epoch 2, Loss: 0.6921, Accuracy: 0.5032
Epoch 3, Loss: 0.6919, Accuracy: 0.5113
Epoch 4, Loss: 0.6917, Accuracy: 0.5117
Epoch 5, Loss: 0.6915, Accuracy: 0.5153
Epoch 6, Loss: 0.6914, Accuracy: 0.5199
Epoch 7, Loss: 0.6912, Accuracy: 0.5267
Epoch 8, Loss: 0.6910, Accuracy: 0.5278
Epoch 9, Loss: 0.6909, Accuracy: 0.5338
Epoch 10, Loss: 0.6908, Accuracy: 0.5313
Epoch 11, Loss: 0.6906, Accuracy: 0.5351
Epoch 12, Loss: 0.6905, Accuracy: 0.5360
Epoch 13, Loss: 0.6903, Accuracy: 0.5351
Epoch 14, Loss: 0.6902, Accuracy: 0.5357
Epoch 15, Loss: 0.6901, Accuracy: 0.5354
Epoch 16, Loss: 0.6900, Accuracy: 0.5352
Epoch 17, Loss: 0.6899, Accuracy: 0.5346
Epoch 18, Loss: 0.6898, Accuracy: 0.5348
Epoch 19, Loss: 0.6897, Accuracy: 0.5349
Epoch 20, Loss: 0.6896, Accuracy: 0.5351


In [14]:
with torch.no_grad():
    print("Day | Predicted | Actual | Probability")
    print("-----------------------------------------")

    correct = 0  # Counter for correct predictions
    total = 100  # Number of days to evaluate

    for i in range(-110, -10):  # Loop over the last 100 predictions
        last_10 = close_prices[i:i+10]  # Past 10 days
        true_today = close_prices[i+9]
        true_tomorrow = close_prices[i+10]

        label = 1 if true_tomorrow > true_today else 0  # Ground truth

        scaled = scaler.transform([last_10])  # Standardize
        input_tensor = torch.tensor(scaled, dtype=torch.float32)
        prob = model(input_tensor).item()
        pred = 1 if prob > 0.5 else 0

        pred_text = "Up" if pred == 1 else "Down"
        label_text = "Up" if label == 1 else "Down"

        if pred == label:
            correct += 1

        print(f"Day {i+111:>3} | Predicted: {pred_text:<4} | Actual: {label_text:<4} | Prob: {prob:.4f}")

    accuracy = correct / total
    print("\nOverall Accuracy: {:.2f}%".format(accuracy * 100))


Day | Predicted | Actual | Probability
-----------------------------------------
Day   1 | Predicted: Down | Actual: Up   | Prob: 0.4903
Day   2 | Predicted: Down | Actual: Down | Prob: 0.4819
Day   3 | Predicted: Down | Actual: Down | Prob: 0.4934
Day   4 | Predicted: Down | Actual: Down | Prob: 0.4862
Day   5 | Predicted: Down | Actual: Up   | Prob: 0.4961
Day   6 | Predicted: Down | Actual: Down | Prob: 0.4960
Day   7 | Predicted: Down | Actual: Up   | Prob: 0.4948
Day   8 | Predicted: Down | Actual: Down | Prob: 0.4899
Day   9 | Predicted: Down | Actual: Up   | Prob: 0.4894
Day  10 | Predicted: Down | Actual: Down | Prob: 0.4837
Day  11 | Predicted: Down | Actual: Down | Prob: 0.4837
Day  12 | Predicted: Down | Actual: Up   | Prob: 0.4819
Day  13 | Predicted: Down | Actual: Up   | Prob: 0.4868
Day  14 | Predicted: Down | Actual: Down | Prob: 0.4788
Day  15 | Predicted: Down | Actual: Up   | Prob: 0.4738
Day  16 | Predicted: Down | Actual: Down | Prob: 0.4710
Day  17 | Predicted: Do